<span style="color:#ffd33d; font-size:150%"> 2023.07.20: DLinear Model Demo </span>
## 2023.07.13
BASE) _Kick_Off_
## 2023.07.17
BASE)_BIVA
 - model edit
 - BIVA-RITS, BIVA-BRITS, BIVA-VAE, BIVA,
## 2023.07.19
 - main, datalorder
## 2023.07.20
 - ipynb


In [13]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/ZZ/Code_02/model/

In [14]:
# !git clone https://github.com/hugwolf77/SVU_SCT_FIN.git

In [15]:
# %cd /content/SVU_SCT_FIN/Model/BIVA/model/

SVU_SCT_FIN/Model/BIVA/model/

In [3]:
# import argparse
import os
import sys
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
import io
from pickle import FALSE

from datetime import date, datetime
import time
from dataclasses import dataclass, field

from typing import Set, List, Dict, Tuple #, final

import torch
import random
import numpy as np
import pandas as pd
from pickle import FALSE

from model.exp_main import Exp_Main
# from model.exp_stat import Exp_Main as exp_stats
from model.utils.metrics import *
from sklearn.preprocessing import StandardScaler
# from scipy.stats import pearsonr

# from sklearn.metrics import *

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  
import matplotlib.font_manager as fm

import seaborn as sns
import IPython
import IPython.display
mpl.rcParams['figure.figsize'] = (16,12)
mpl.rcParams['axes.grid'] = False
# %matplotlib inline

2025-06-03 13:42:44.601008: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 13:42:44.731735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-03 13:42:44.797389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-03 13:42:44.819743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 13:42:44.921211: I tensorflow/core/platform/cpu_feature_guar

In [4]:
cwd = os.path.dirname(os.getcwd())
cwd

'/home/augustine77/mylab/sim/sim/05_SVU_SCT_FIN/Model/BIVA'

In [7]:

fix_seed = 2025
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

# argparser 에서 class 형태로 변경.
class args:
    pass

save_path = '/home/augustine77/mylab/sim/sim/05_SVU_SCT_FIN/Model/BIVA/exp'

#-----------------------------------------------------------------------------------------------------------------------
result_df = pd.DataFrame(columns=['no','model_id','root','len_seq','len_pred',
                                  'learn_rate','batch','MSE','MAE','RMSE','MAPE'])
no = 0

len_seq_list = [6] #,9,12,15,18,21,24]
len_pred_list = [1] #,2,3,4]
batch_list = [5]
lr_rate_list = [0.5]

#-----------------------------------------------------------------------------------------------------------------------

for len_seq in len_seq_list:
  for len_pred in len_pred_list:
    for batch in batch_list:
      for lr_rate in lr_rate_list:
        no += 1

        now = time.localtime()
        time_now = time.strftime('%Y%m%d', now)

        args.is_training = True    # 'status'
        args.model = 'BIVA'  # 'model name'
        args.model_id = '{}-no{}-dt{}-seq{}-pred{}-bt{}-lr{}-mv6'.format(args.model,no,time_now,len_seq,len_pred,batch,lr_rate)

        # dataset
        args.data = 'BIVA'                                                                                             # 'dataset type : data_loader'
        args.root_path = '/home/augustine77/mylab/sim/sim/05_SVU_SCT_FIN/Model/BIVA/Base_data'
        args.data_path = 'dataset_03_S.xlsx'

        args.des = 'Exp'                                                                                                 # 'exp description'
        args.loss = 'mse'                                                                                                # loss function
        args.lradj = '1' #'dtype1'                                                                                             # adjusting learning rate option
        args.use_amp = False                                                                                             # 'use automatic mixed precision training'

        args.features = 'MS'                                                                                             # 'forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate'
        args.target = 'A1' # 'GDP'                                                                                       # 'target feature in S or MS task')
        args.freq = 'M'                                                                                                  # 'freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h'
        args.checkpoints =  '/home/augustine77/mylab/sim/sim/05_SVU_SCT_FIN/Model/BIVA/exp/(BIVA)_checkpoints/'                                   #'location of model checkpoints'

        # forecasting task
        args.seq_len = len_seq                                                                                           # 'input sequence length'
        args.pred_len = len_pred
        args.label_len = 1

        args.channels = 102
        args.rnn_hid_size = 256
        args.vae_hid_size = 256    ## deprivation
        args.vae_latent_size = 14
        args.infer_hid_size = 128
        args.moving_avg = 6                                                                                    # 'window size of moving average'
        # args.moving_avg = [6,10,15,20,25,30,35,40,45,60]                                                              # 'window size' list of moving average'


        args.cols = None

        #
        args.conv1d = True
        args.conv_kernal = 1
        args.RIN = False
        args.combination = True

        args.imputed_weight = 0.5

        args.last_chkpt = None #'BIVA-no1-dt20230801-seq6-pred1-bt5-lr0.5-mv6_last_checkpoint.pth'
        args.learning_rate = lr_rate
        args.batch_size = batch
        args.train_epochs = 2           # 'train epochs'
        args.patience = 10              # 'early stopping patience'

        args.embed = 'timeF'            # 'time features encoding, options:[timeF, fixed, learned]'
        args.inverse = False
        # args.activation = 'relu'        # 'activation'
        args.do_predict = False         # 'whether to predict unseen future data')

        # optimization
        args.num_workers = 10            # 'data loader num workers'
        args.itr = 1                    # 'experiments times'

        # GPU
        args.use_gpu = False             # 'use gpu'
        args.gpu = 0                    # 'gpu'
        args.use_multi_gpu = False      # 'use multiple gpus'
        args.devices = '0,1,2,3'        # 'device ids of multile gpus'
        # args.test_flop = False          # 'See utils/tools for usage'

        args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False
        if args.use_gpu and args.use_multi_gpu:
            args.dvices = args.devices.replace(' ', '')
            device_ids = args.devices.split(',')
            args.device_ids = [int(id_) for id_ in device_ids]
            args.gpu = args.device_ids[0]

        print('Args in experiment:')
        print(args)
        #-----------------------------------------------------------------------------------------------------------------------
        print(f"\n train start -----------------------> {no}\n")
        #-----------------------------------------------------------------------------------------------------------------------

        if args.is_training:
            start_time = time.time()
            for ii in range(args.itr):
                # setting record of experiments
                setting = '{}_{}_seq{}_pred{}_bt{}_lr{}_{}'.format(
                    args.model_id,
                    args.data,
                    args.seq_len,
                    args.pred_len,
                    args.batch_size,
                    args.learning_rate,
                    args.channels,
                    args.des, ii)

                exp = Exp_Main(args)  # set experiments
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                preds, trues, inputx, mae, mse, rmse, mape, mspe, rse, corr = exp.test(setting)

                if args.do_predict:
                    print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                    exp.predict(setting, True)

                time_cost = str(time.time() - start_time)

                result = {'no':no+1,'Model_id':args.model_id,'root':args.root_path,'len_seq':args.seq_len,'len_pred':args.pred_len,
                          'learn_rate':args.learning_rate,'batch':args.batch_size,
                          'MSE':mse,'MAE':mae,'RMSE':rmse, 'MAPE':mape,'Corr':corr, 'time_cost': time_cost}
                torch.cuda.empty_cache()
        else:
            ii = 0
            setting = '{}_{}_seq{}_pred{}_bt{}_lr{}_{}'.format(
                    args.model_id,
                    args.data,
                    args.seq_len,
                    args.pred_len,
                    args.batch_size,
                    args.learning_rate,
                    args.channels,
                    args.des, ii)

            exp = Exp_Main(args)  # set experiments
            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            preds, trues, inputx, mae, mse, rmse, mape, mspe, rse, corr = exp.test(setting, test=1)

            result = {'no':no+1,'Model_id':args.model_id,'root':args.root_path,'len_seq':args.seq_len,'len_pred':args.pred_len,
                          'learn_rate':args.learning_rate,'batch':args.batch_size,
                          'MSE':mse,'MAE':mae,'RMSE':rmse, 'MAPE':mape,'Corr':corr, 'time_cost': time_cost}
            torch.cuda.empty_cache()


        result_df = pd.concat([result_df,pd.DataFrame(result)], ignore_index = True)
        result_df.to_csv(os.path.join(save_path,'(BIVA)_{}_CV_no{}_exp01.csv'.format(args.model_id,no)))
        print(f"\n train done -----------------------> {no} ---- {time_cost} \n")
        no += 1



Args in experiment:
<class '__main__.args'>

 train start -----------------------> 1

Use CPU
>>>>>>>start training : BIVA-no1-dt20250603-seq6-pred1-bt5-lr0.5-mv6_BIVA_seq6_pred1_bt5_lr0.5_102>>>>>>>>>>>>>>>>>>>>>>>>>>
train 220
val 30
test 29


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!